# ChatBot APP


**Note**: In June 2023, OpenAI updated gpt-3.5-turbo. The results you see in the notebook may be slightly different than those in the video. Some of the prompts have also been slightly modified to product the desired results.

## Setup
Let's import necessary libraries: os, openai, and dotenv.

Then, we load environment variables from a .env file using load_dotenv and find_dotenv functions from the dotenv library.

Finally, it sets the API key for the OpenAI library by retrieving it from the environment variables using `os.getenv('OPENAI_API_KEY')`. This API key allows access to OpenAI's services.

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

We have two functions here. The first one, `get_completion`, takes a prompt and an optional model parameter, which defaults to "gpt-3.5-turbo". It creates a list of messages with a user role containing the provided prompt. Then, it uses OpenAI's class `ChatCompletion` to generate a response based on the given prompt. The `temperature` parameter controls the randomness of the model's output. Finally, it returns the content of the generated response.

In the second function, `get_completion_from_messages`, instead of kind of putting a single prompt as input and getting a single completion, we're going to pass in a list of messages. And these messages can be kind of from a variety of different roles, so Let's describe those.

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

Let's try to use these messages in a conversation

In [ ]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [3]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

**Note**      
Each Conversation with a language model as a standalone interaction, which means that you must provide all relevant messages for the model to draw from in the current conversation. 

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},    
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

## OrderBot
Going to build your own chatbot.This chat bot is going to be called Autobot.And.We're going to automate the collection of user prompts and assistant responses in order to build this order box.And it's going to take orders at a pizza restaurant.

So first we're going to define `collect_messages()` function. And what this is doing is it's going to kind of collect our user messages so we can avoid typing them in by hand in the way that we did above. And this is going to kind of collect prompts from a user interface that we'll build below.
And then append it to a list called context and then it will call the model with that context every time, the royal response is then also added to the contact. So the kind of model message is added to the context, the user message is added to the context, so on. So it just kind of grows longer and longer.
This way the model has the information it needs to determine what to do next.

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

And so here's the context, and it contains the system message that contains the menu.     
**Note**    
Every time we call the language model, we're going to use the same context, and the context is building up overtime

In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

And so now we can ask the model to create a JSON summary that we could send to the order system based on the conversation.So we're now appending another system message which is an instruction and we're saying create a JSON summary of the previous food order itemize the price for each item.

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

And notice in this case we're using lower temperature because for these kinds of tasks we want the output to be fairly predictable. For a conversational agent you might want to use a higher temperature. However in this case I would maybe use a lower temperature as well, because for our customers assistant shop what you might want the output to be a bit more predictable as well.